In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from ultralytics import YOLO
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm
from torchvision import transforms
import multiprocessing
import sys

In [3]:
device = torch.device('cuda')
print('Using device:', device)

# Load YOLOv8
yolo_model = YOLO('models/yolov8n.pt').to(device)

Using device: cuda


In [4]:
def detect_faces_yolo(img_path, model):
    img = cv2.imread(img_path)
    results = model(img)
    boxes = results[0].boxes.xyxy.cpu().numpy()
    faces = []
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        face = img[y1:y2, x1:x2]
        if face.size > 0:
            faces.append(cv2.resize(face, (256, 256)))
    return faces

In [5]:
def load_image_paths(folder, label, max_samples=2000):
    paths, labels = [], []
    count = 0
    for file in os.listdir(folder):
        if count >= max_samples:
            break
        path = os.path.join(folder, file)
        if os.path.isfile(path):
            paths.append(path)
            labels.append(label)
            count += 1
    return paths, labels

In [6]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  std=[0.229, 0.224, 0.225]),
])

In [7]:
class DeepfakeDataset(Dataset):
    def __init__(self, paths, labels, model, transform=None, extract_features=False):
        self.paths = paths
        self.labels = labels
        self.model = model
        self.transform = transform
        self.extract_features = extract_features

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = cv2.imread(self.paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        if self.extract_features:
            faces = detect_faces_yolo(self.paths[idx], self.model)
            if len(faces) > 0:
                img = faces[0]

        if self.transform:
            img = self.transform(img)

        return img, label

In [8]:
real_paths, real_labels = load_image_paths('data/train/real', 0, max_samples=2000)
fake_paths, fake_labels = load_image_paths('data/train/fake', 1, max_samples=2000)

all_paths = real_paths + fake_paths
all_labels = real_labels + fake_labels

X_train, X_val, y_train, y_val = train_test_split(all_paths, all_labels, test_size=0.2, random_state=42)

In [15]:
train_ds = DeepfakeDataset(X_train, y_train, yolo_model, transform=train_transform, extract_features=True)
val_ds = DeepfakeDataset(X_val, y_val, yolo_model, transform=val_transform, extract_features=True)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size=32, num_workers=0, pin_memory=True)

In [21]:
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Sequential(nn.Linear(model._fc.in_features, 1))
model = model.to(device)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)
scaler = GradScaler()

Loaded pretrained weights for efficientnet-b0


C:\Users\shruj\AppData\Local\Temp\ipykernel_22644\1208028892.py:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [28]:
def calculate_accuracy(preds, labels):
    preds = (preds > 0.5).float()
    correct = (preds == labels).sum().item()
    return correct / labels.size(0)

In [23]:
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    total_acc = 0.0
    processed_images = 0

    for xb, yb in tqdm(train_dl):
        xb, yb = xb.to(device), yb.to(torch.float32).unsqueeze(1).to(device)
        optimizer.zero_grad()
        with autocast():
            preds = model(xb)
            loss = loss_fn(preds, yb)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        batch_size = xb.size(0)
        processed_images += batch_size
        total_loss += loss.item()
        total_acc += calculate_accuracy(preds, yb) * batch_size

    print(f"Epoch {epoch+1}: Train Loss = {total_loss/len(train_dl):.4f}, Train Acc = {total_acc/len(train_ds):.4f}")

    model.eval()
    val_loss, val_acc = 0.0, 0.0
    with torch.no_grad():
        for xb, yb in val_dl:
            xb, yb = xb.to(device), yb.to(torch.float32).unsqueeze(1).to(device)
            preds = model(xb)
            loss = loss_fn(preds, yb)
            val_loss += loss.item()
            val_acc += calculate_accuracy(preds, yb) * xb.size(0)

    print(f"Epoch {epoch+1}: Val Loss = {val_loss/len(val_dl):.4f}, Val Acc = {val_acc/len(val_ds):.4f}")
    scheduler.step(val_loss / len(val_dl))

0: 640x640 2 persons, 210.2ms
Speed: 1.9ms preprocess, 210.2ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 204.9ms
Speed: 4.0ms preprocess, 204.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 persons, 1 bottle, 203.4ms
Speed: 2.0ms preprocess, 203.4ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 apple, 204.0ms
Speed: 2.0ms preprocess, 204.0ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 202.9ms
Speed: 3.0ms preprocess, 202.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 203.3ms
Speed: 3.0ms preprocess, 203.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 211.7ms
Speed: 4.0ms preprocess, 211.7ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 1 clock, 215.3ms
Speed: 3.1ms preprocess, 215.3ms inference, 

 24%|██▍       | 24/100 [03:30<10:56,  8.64s/it]


0: 640x640 5 persons, 206.3ms
Speed: 2.4ms preprocess, 206.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 203.4ms
Speed: 2.0ms preprocess, 203.4ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 198.5ms
Speed: 3.0ms preprocess, 198.5ms inference, 5.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 persons, 201.4ms
Speed: 2.0ms preprocess, 201.4ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 203.3ms
Speed: 1.2ms preprocess, 203.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 199.4ms
Speed: 5.9ms preprocess, 199.4ms inference, 5.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 203.1ms
Speed: 0.6ms preprocess, 203.1ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 212.5ms
Speed: 2.6ms preprocess, 212.5ms inference, 0.0ms postprocess per imag

 25%|██▌       | 25/100 [03:38<10:45,  8.60s/it]


0: 640x640 2 persons, 239.3ms
Speed: 0.0ms preprocess, 239.3ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 persons, 207.9ms
Speed: 3.0ms preprocess, 207.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 212.6ms
Speed: 3.1ms preprocess, 212.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 210.3ms
Speed: 3.0ms preprocess, 210.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 211.2ms
Speed: 3.0ms preprocess, 211.2ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 tie, 207.6ms
Speed: 3.0ms preprocess, 207.6ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 205.2ms
Speed: 2.0ms preprocess, 205.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 203.5ms
Speed: 3.0ms preprocess, 203.5ms inference, 2.0ms postpro

...

 99%|█████████▉| 99/100 [14:09<00:08,  8.80s/it]


0: 640x640 1 person, 200.7ms
Speed: 3.2ms preprocess, 200.7ms inference, 6.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 206.3ms
Speed: 2.6ms preprocess, 206.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 202.1ms
Speed: 2.1ms preprocess, 202.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 199.9ms
Speed: 2.5ms preprocess, 199.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 203.0ms
Speed: 2.0ms preprocess, 203.0ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1 laptop, 202.7ms
Speed: 2.1ms preprocess, 202.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 201.8ms
Speed: 3.0ms preprocess, 201.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 202.0ms
Speed: 2.0ms preprocess, 202.0ms inference, 4.0ms postproce

100%|██████████| 100/100 [14:18<00:00,  8.59s/it]

Epoch 10: Train Loss = 0.0273, Train Acc = 0.9881



0: 640x640 2 persons, 210.6ms
Speed: 4.0ms preprocess, 210.6ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 3 apples, 1 orange, 202.6ms
Speed: 2.3ms preprocess, 202.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 1 tie, 202.5ms
Speed: 2.0ms preprocess, 202.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1 frisbee, 202.3ms
Speed: 2.0ms preprocess, 202.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 202.4ms
Speed: 2.0ms preprocess, 202.4ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 persons, 202.7ms
Speed: 3.3ms preprocess, 202.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 1 tie, 202.2ms
Speed: 2.0ms preprocess, 202.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 201.3ms
Speed: 2.0ms preprocess, 

In [24]:
torch.save(model.state_dict(), 'models/efficientnet_model.pt')

In [25]:
def extract_deep_features(paths, model, device, yolo_model, size=(256, 256)):
    model.eval()
    features = []
    with torch.no_grad():
        for path in tqdm(paths, desc="Extracting deep features"):
            img = cv2.imread(path)
            results = yolo_model(img)
            boxes = results[0].boxes.xyxy.cpu().numpy()

            if len(boxes) > 0:
                x1, y1, x2, y2 = map(int, boxes[0])
                face = img[y1:y2, x1:x2]
                if face.size > 0:
                    img = cv2.resize(face, size)
                else:
                    img = cv2.resize(img, size)
            else:
                img = cv2.resize(img, size)

            img = img[:, :, ::-1].copy()
            img_tensor = torch.tensor(img).permute(2, 0, 1).unsqueeze(0).float().to(device) / 255.0

            feat = model.extract_features(img_tensor)
            feat = torch.nn.functional.adaptive_avg_pool2d(feat, (1, 1))
            feat = feat.view(-1).cpu().numpy()
            features.append(feat)

    return np.array(features)

In [26]:
X_train_feats = extract_deep_features(X_train, model, device, yolo_model)
X_val_feats = extract_deep_features(X_val, model, device, yolo_model)

0: 640x640 1 person, 208.0ms
Speed: 2.4ms preprocess, 208.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


Extracting deep features:  98%|█████████▊| 781/800 [03:19<00:04,  3.85it/s]


0: 640x640 2 persons, 208.0ms
Speed: 4.3ms preprocess, 208.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


Extracting deep features:  98%|█████████▊| 782/800 [03:19<00:04,  3.89it/s]


0: 640x640 1 person, 208.6ms
Speed: 3.0ms preprocess, 208.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


...

Extracting deep features: 100%|█████████▉| 798/800 [03:23<00:00,  3.93it/s]


0: 640x640 2 persons, 205.7ms
Speed: 2.0ms preprocess, 205.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


Extracting deep features: 100%|█████████▉| 799/800 [03:23<00:00,  3.93it/s]


0: 640x640 1 person, 207.0ms
Speed: 3.0ms preprocess, 207.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


Extracting deep features: 100%|██████████| 800/800 [03:23<00:00,  3.92it/s]


In [27]:
xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='logloss',
    tree_method='gpu_hist' if torch.cuda.is_available() else 'auto'
)

xgb.fit(X_train_feats, y_train, eval_set=[(X_val_feats, y_val)], early_stopping_rounds=10, verbose=True)

preds = xgb.predict(X_val_feats)
print("XGBoost Accuracy:", accuracy_score(y_val, preds))
xgb.save_model('models/xgboost_model.json')

C:\Users\shruj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\shruj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\core.py:160: UserWarning: [14:51:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[0]	validation_0-logloss:0.66184
[1]	validation_0-logloss:0.63199
[2]	validation_0-logloss:0.60441
[3]	validation_0-logloss:0.57897
[4]	validation_0-logloss:0.55595
[5]	validation_0-logloss:0.53392
[6]	validation_0-logloss:0.51467
[7]	validation_0-logloss:0.49652
[8]	validation_0-logloss:0.47971
[9]	validation_0-logloss:0.46459
[10]	validation_0-logloss:0.45003
[11]	validation_0-logloss:0.43617
[12]	validation_0-logloss:0.42380
[13]	validation_0-logloss:0.41277
[14]	validation_0-logloss:0.40252
[15]	validation_0-logloss:0.39176
[16]	validation_0-logloss:0.38204
[17]	validation_0-logloss:0.37285
[18]	validation_0-logloss:0.36460
[19]	validation_0-logloss:0.35673
[20]	validation_0-logloss:0.34890
[21]	validation_0-logloss:0.34207
[22]	validation_0-logloss:0.33605
[23]	validation_0-logloss:0.32950
[24]	validation_0-logloss:0.32392
[25]	validation_0-logloss:0.31884
[26]	validation_0-logloss:0.31368
[27]	validation_0-logloss:0.30882
[28]	validation_0-logloss:0.30430
[29]	validation_0-loglos

C:\Users\shruj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\core.py:160: UserWarning: [14:52:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\shruj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\core.py:160: UserWarning: [14:52:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower perform